In [1]:
!unrar x "/content/BachelorData.rar" "/content/BachelorData2/"

Streaming af output blev afkortet til de sidste 5000 linjer.
Extracting  /content/BachelorData2/BachelorData/training/pneumonia/person348_virus_716.jpeg      77%  OK 
Extracting  /content/BachelorData2/BachelorData/training/pneumonia/person348_virus_716_aug4885.jpeg      77%  OK 
Extracting  /content/BachelorData2/BachelorData/training/pneumonia/person348_virus_716_aug9158.jpeg      77%  OK 
Extracting  /content/BachelorData2/BachelorData/training/pneumonia/person348_virus_717.jpeg      77%  OK 
Extracting  /content/BachelorData2/BachelorData/training/pneumonia/person348_virus_717_aug6247.jpeg      77%  OK 
Extracting  /content/BachelorData2/BachelorData/training/pneumonia/person348_virus_719.jpeg      77%  OK 
Extracting  /content/BachelorData2/BachelorData/training/pneumonia/person348_virus_719_aug6868.jpeg      77%  OK 
Extracting  /content/BachelorData2/BachelorData/training/pneumonia/person348_virus_720.jpeg      7

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from keras.callbacks import Callback

# Custom Keras callback for additional metrics
class Metrics(Callback):
    def __init__(self, val_generator):
        super().__init__()
        self.val_generator = val_generator

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = (np.asarray(self.model.predict(self.val_generator))).round()
        val_true = self.val_generator.classes
        _val_f1 = f1_score(val_true, val_predict)
        _val_roc_auc = roc_auc_score(val_true, val_predict)
        _val_specificity = confusion_matrix(val_true, val_predict)[0, 0] / (
            confusion_matrix(val_true, val_predict)[0, 0] + confusion_matrix(val_true, val_predict)[0, 1]
        )
        logs['val_f1'] = _val_f1
        logs['val_roc_auc'] = _val_roc_auc
        logs['val_specificity'] = _val_specificity
        print(f" - val_f1: {_val_f1} - val_roc_auc: {_val_roc_auc} - val_specificity: {_val_specificity}")
        return

metrics = Metrics(validation_generator)

# LeNet-5 model
model = Sequential()
model.add(Conv2D(6, (5, 5), activation='relu', input_shape=(200, 200, 3)))
model.add(AveragePooling2D())
model.add(Conv2D(16, (5, 5), activation='relu'))
model.add(AveragePooling2D())
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])

# Load and preprocess the data
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    '/content/BachelorData2/BachelorData/training',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/BachelorData2/BachelorData/validation',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary'
)

# Train the model
epochs = 10
steps_per_epoch = 20000 // 64
validation_steps = 500 // 64

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[metrics],
)

# Load the test data
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    '/content/BachelorData2/BachelorData/test',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

# Evaluate the model on the test set
test_metrics = model.evaluate(test_generator)
print(f"Test loss: {test_metrics[0]}, Test accuracy: {test_metrics[1]}, Test Precision: {test_metrics[2]}, Test Recall: {test_metrics[3]}")


# Calculate additional metrics on the test set
test_true = test_generator.classes
test_predict = (model.predict(test_generator)).round()

_test_f1 = f1_score(test_true, test_predict)
_test_roc_auc = roc_auc_score(test_true, test_predict)
_test_specificity = confusion_matrix(test_true, test_predict)[0, 0] / (confusion_matrix(test_true, test_predict)[0, 0] + confusion_matrix(test_true, test_predict)[0, 1])

print(f"Test F1 score: {_test_f1}")
print(f"Test AUC-ROC: {_test_roc_auc}")
print(f"Test specificity: {_test_specificity}")

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 20000 images belonging to 2 classes.
Found 587 images belonging to 2 classes.
Epoch 1/10
10/10 [==============================] - 1s 78ms/step
 - val_f1: 0.7953586497890296 - val_roc_auc: 0.4907350261564686 - val_specificity: 0.10062893081761007
312/312 [==============================] - 23s 70ms/step - loss: 2.5192 - accuracy: 0.8087 - precision_2: 0.8085 - recall_2: 0.8092 - val_loss: 0.8578 - val_accuracy: 0.8304 - val_precision_2: 0.8142 - val_recall_2: 0.9907 - val_f1: 0.7954 - val_roc_auc: 0.4907 - val_specificity: 0.1006
Epoch 2/10
10/10 [==============================] - 1s 77ms/step
 - val_f1: 0.4334975369458128 - val_roc_auc: 0.5001175571621701 - val_specificity: 0.6918238993710691
312/312 [==============================] - 22s 69ms/step - loss: 0.4337 - accuracy: 0.8757 - precision_2: 0.8780 - recall_2: 0.8726 - val_loss: 1.4915 - val_accuracy: 0.5603 - val_precision_2: 0.9925 - val_recall_2: 0.4024 - val_f1: 0.4335 - val_roc_auc: 0.5001 - val_specificity: 0.6918
Epoch

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix

class Metrics(Callback):
    def __init__(self, val_generator):
        super().__init__()
        self.val_generator = val_generator

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = (np.asarray(self.model.predict(self.val_generator))).round()
        val_true = self.val_generator.classes
        _val_f1 = f1_score(val_true, val_predict)
        _val_roc_auc = roc_auc_score(val_true, val_predict)
        _val_specificity = confusion_matrix(val_true, val_predict)[0, 0] / (
            confusion_matrix(val_true, val_predict)[0, 0] + confusion_matrix(val_true, val_predict)[0, 1]
        )
        logs['val_f1'] = _val_f1
        logs['val_roc_auc'] = _val_roc_auc
        logs['val_specificity'] = _val_specificity
        print(f" - val_f1: {_val_f1} - val_roc_auc: {_val_roc_auc} - val_specificity: {_val_specificity}")
        return

# Define AlexNet model
model = Sequential([
    Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(200, 200, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])

# Load and preprocess the data
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    '/content/BachelorData2/BachelorData/training',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/BachelorData2/BachelorData/validation',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary'
)

# Train the model
epochs = 10
steps_per_epoch = 20000 // 64
validation_steps = 500 // 64

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[metrics],
)

# Load the test data
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    '/content/BachelorData2/BachelorData/test',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

# Evaluate the model on the test set
test_metrics = model.evaluate(test_generator)
print(f"Test loss: {test_metrics[0]}, Test accuracy: {test_metrics[1]}, Test Precision: {test_metrics[2]}, Test Recall: {test_metrics[3]}")


# Calculate additional metrics on the test set
test_true = test_generator.classes
test_predict = (model.predict(test_generator)).round()

_test_f1 = f1_score(test_true, test_predict)
_test_roc_auc = roc_auc_score(test_true, test_predict)
_test_specificity = confusion_matrix(test_true, test_predict)[0, 0] / (confusion_matrix(test_true, test_predict)[0, 0] + confusion_matrix(test_true, test_predict)[0, 1])

print(f"Test F1 score: {_test_f1}")
print(f"Test AUC-ROC: {_test_roc_auc}")
print(f"Test specificity: {_test_specificity}")


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 20000 images belonging to 2 classes.
Found 587 images belonging to 2 classes.
Epoch 1/10
10/10 [==============================] - 1s 99ms/step
 - val_f1: 0.7653508771929824 - val_roc_auc: 0.4831819784870393 - val_specificity: 0.1509433962264151
312/312 [==============================] - 23s 71ms/step - loss: 0.4166 - accuracy: 0.8742 - precision_1: 0.8773 - recall_1: 0.8701 - val_loss: 0.4645 - val_accuracy: 0.7835 - val_precision_1: 0.8224 - val_recall_1: 0.9039 - val_f1: 0.7654 - val_roc_auc: 0.4832 - val_specificity: 0.1509
Epoch 2/10
10/10 [==============================] - 1s 75ms/step
 - val_f1: 0.7063020214030915 - val_roc_auc: 0.4821827426085935 - val_specificity: 0.27044025157232704
312/312 [==============================] - 21s 67ms/step - loss: 0.2215 - accuracy: 0.9180 - precision_1: 0.9231 - recall_1: 0.9119 - val_loss: 0.2029 - val_accuracy: 0.9152 - val_precision_1: 0.9571 - val_recall_1: 0.9286 - val_f1: 0.7063 - val_roc_auc: 0.4822 - val_specificity: 0.2704
Epoch

In [ ]:
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.metrics import Precision, Recall
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from keras.callbacks import Callback


class Metrics(Callback):
    def __init__(self, val_generator):
        super().__init__()
        self.val_generator = val_generator

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = (np.asarray(self.model.predict(self.val_generator))).round()
        val_true = self.val_generator.classes
        _val_f1 = f1_score(val_true, val_predict)
        _val_roc_auc = roc_auc_score(val_true, val_predict)
        _val_specificity = confusion_matrix(val_true, val_predict)[0, 0] / (
            confusion_matrix(val_true, val_predict)[0, 0] + confusion_matrix(val_true, val_predict)[0, 1]
        )
        logs['val_f1'] = _val_f1
        logs['val_roc_auc'] = _val_roc_auc
        logs['val_specificity'] = _val_specificity
        print(f" - val_f1: {_val_f1} - val_roc_auc: {_val_roc_auc} - val_specificity: {_val_specificity}")
        return



# Load the pre-trained VGG-19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 1024 units and ReLU activation
x = Dense(1024, activation='relu')(x)

# Add a final output layer for binary classification
predictions = Dense(1, activation='sigmoid')(x)

# Construct the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Set all layers in the base model to be non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])

# Load and preprocess the data
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    '/content/BachelorData2/BachelorData/training',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/BachelorData2/BachelorData/validation',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary'
)

metrics = Metrics(validation_generator)

# Train the model
epochs = 20
steps_per_epoch = 20000 // 64
validation_steps = 500 // 64

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[metrics],
)

# Load the test data
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    '/content/BachelorData2/BachelorData/test',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

# Evaluate the model on the test set
test_metrics = model.evaluate(test_generator)
print(f"Test loss: {test_metrics[0]}, Test accuracy: {test_metrics[1]}, Test Precision: {test_metrics[2]}, Test Recall: {test_metrics[3]}")


# Calculate additional metrics on the test set
test_true = test_generator.classes
test_predict = (model.predict(test_generator)).round()

_test_f1 = f1_score(test_true, test_predict)
_test_roc_auc = roc_auc_score(test_true, test_predict)
_test_specificity = confusion_matrix(test_true, test_predict)[0, 0] / (confusion_matrix(test_true, test_predict)[0, 0] + confusion_matrix(test_true, test_predict)[0, 1])

print(f"Test F1 score: {_test_f1}")
print(f"Test AUC-ROC: {_test_roc_auc}")
print(f"Test specificity: {_test_specificity}")


80134624/80134624 [==============================] - 4s 0us/step


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 20000 images belonging to 2 classes.
Found 587 images belonging to 2 classes.
Epoch 1/20
10/10 [==============================] - 1s 118ms/step
 - val_f1: 0.5689900426742531 - val_roc_auc: 0.4977958032093105 - val_specificity: 0.5283018867924528
312/312 [==============================] - 34s 75ms/step - loss: 0.2139 - accuracy: 0.9154 - precision: 0.9171 - recall: 0.9133 - val_loss: 0.7256 - val_accuracy: 0.7165 - val_precision: 0.9758 - val_recall: 0.6235 - val_f1: 0.5690 - val_roc_auc: 0.4978 - val_specificity: 0.5283
Epoch 2/20
10/10 [==============================] - 1s 78ms/step
 - val_f1: 0.6276041666666666 - val_roc_auc: 0.4702213013577852 - val_specificity: 0.37735849056603776
312/312 [==============================] - 21s 69ms/step - loss: 0.1387 - accuracy: 0.9479 - precision: 0.9497 - recall: 0.9459 - val_loss: 0.4146 - val_accuracy: 0.8214 - val_precision: 0.9766 - val_recall: 0.7716 - val_f1: 0.6276 - val_roc_auc: 0.4702 - val_specificity: 0.3774
Epoch 3/20
10/10 [==

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Load the ResNet-50 model without the top layers
base_model = ResNet50(weights=None, include_top=False, input_shape=(200, 200, 3))

# Add a global average pooling layer and a dense layer for the binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[Metrics(test_generator)]
)

# Evaluate the model on the test set
test_metrics = model.evaluate(test_generator)
print(f"Test loss: {test_metrics[0]}, Test accuracy: {test_metrics[1]}, Test Precision: {test_metrics[2]}, Test Recall: {test_metrics[3]}")

# Calculate additional metrics on the test set
test_true = test_generator.classes
test_predict = (model.predict(test_generator)).round()

_test_f1 = f1_score(test_true, test_predict)
_test_roc_auc = roc_auc_score(test_true, test_predict)
_test_specificity = confusion_matrix(test_true, test_predict)[0, 0] / (confusion_matrix(test_true, test_predict)[0, 0] + confusion_matrix(test_true, test_predict)[0, 1])

print(f"Test F1 score: {_test_f1}")
print(f"Test AUC-ROC: {_test_roc_auc}")
print(f"Test specificity: {_test_specificity}")


Epoch 1/10


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


10/10 [==============================] - 2s 77ms/step
 - val_f1: 0.8433497536945812 - val_roc_auc: 0.5 - val_specificity: 0.0
312/312 [==============================] - 41s 110ms/step - loss: 0.2863 - accuracy: 0.8821 - precision: 0.8841 - recall: 0.8794 - val_loss: 1.2899 - val_accuracy: 0.7054 - val_precision: 0.7054 - val_recall: 1.0000 - val_f1: 0.8433 - val_roc_auc: 0.5000 - val_specificity: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 1s 79ms/step
 - val_f1: 0.023094688221709004 - val_roc_auc: 0.5058411214953271 - val_specificity: 1.0
312/312 [==============================] - 33s 104ms/step - loss: 0.2025 - accuracy: 0.9205 - precision: 0.9222 - recall: 0.9187 - val_loss: 15.2213 - val_accuracy: 0.2812 - val_precision: 1.0000 - val_recall: 0.0031 - val_f1: 0.0231 - val_roc_auc: 0.5058 - val_specificity: 1.0000
Epoch 3/10
10/10 [==============================] - 1s 75ms/step
 - val_f1: 0.004662004662004662 - val_roc_auc: 0.5011682242990654 - val_specificity: 1.0

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the DenseNet-121 model without the top layers
base_model = DenseNet121(include_top=False, weights=None, input_shape=(200, 200, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[Metrics(test_generator)]
)

# Evaluate the model on the test set
test_metrics = model.evaluate(test_generator)
print(f"Test loss: {test_metrics[0]}, Test accuracy: {test_metrics[1]}, Test Precision: {test_metrics[2]}, Test Recall: {test_metrics[3]}")

# Calculate additional metrics on the test set
test_true = test_generator.classes
test_predict = (model.predict(test_generator)).round()

_test_f1 = f1_score(test_true, test_predict)
_test_roc_auc = roc_auc_score(test_true, test_predict)
_test_specificity = confusion_matrix(test_true, test_predict)[0, 0] / (confusion_matrix(test_true, test_predict)[0, 0] + confusion_matrix(test_true, test_predict)[0, 1])

print(f"Test F1 score: {_test_f1}")
print(f"Test AUC-ROC: {_test_roc_auc}")
print(f"Test specificity: {_test_specificity}")

Epoch 1/10
10/10 [==============================] - 3s 88ms/step
 - val_f1: 0.8824742268041237 - val_roc_auc: 0.6415094339622642 - val_specificity: 0.2830188679245283
312/312 [==============================] - 122s 157ms/step - loss: 0.2522 - accuracy: 0.9016 - precision: 0.9031 - recall: 0.8997 - val_loss: 1.0586 - val_accuracy: 0.7857 - val_precision: 0.7736 - val_recall: 1.0000 - val_f1: 0.8825 - val_roc_auc: 0.6415 - val_specificity: 0.2830
Epoch 2/10
10/10 [==============================] - 1s 78ms/step
 - val_f1: 0.3073929961089494 - val_roc_auc: 0.5702771410098161 - val_specificity: 0.9559748427672956
312/312 [==============================] - 42s 135ms/step - loss: 0.1694 - accuracy: 0.9327 - precision: 0.9318 - recall: 0.9337 - val_loss: 6.0766 - val_accuracy: 0.3839 - val_precision: 0.7848 - val_recall: 0.1931 - val_f1: 0.3074 - val_roc_auc: 0.5703 - val_specificity: 0.9560
Epoch 3/10
10/10 [==============================] - 1s 75ms/step
 - val_f1: 0.8526315789473685 - val_ro

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the EfficientNet-B0 model without the top layers
base_model = EfficientNetB0(include_top=False, weights=None, input_shape=(200, 200, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[Metrics(test_generator)]
)

# Evaluate the model on the test set
test_metrics = model.evaluate(test_generator)
print(f"Test loss: {test_metrics[0]}, Test accuracy: {test_metrics[1]}, Test Precision: {test_metrics[2]}, Test Recall: {test_metrics[3]}")

# Calculate additional metrics on the test set
test_true = test_generator.classes
test_predict = (model.predict(test_generator)).round()

_test_f1 = f1_score(test_true, test_predict)
_test_roc_auc = roc_auc_score(test_true, test_predict)
_test_specificity = confusion_matrix(test_true, test_predict)[0, 0] / (confusion_matrix(test_true, test_predict)[0, 0] + confusion_matrix(test_true, test_predict)[0, 1])

print(f"Test F1 score: {_test_f1}")
print(f"Test AUC-ROC: {_test_roc_auc}")
print(f"Test specificity: {_test_specificity}")

Epoch 1/10
10/10 [==============================] - 2s 76ms/step
 - val_f1: 0.8433497536945812 - val_roc_auc: 0.5 - val_specificity: 0.0
312/312 [==============================] - 83s 124ms/step - loss: 0.3763 - accuracy: 0.8357 - precision: 0.8398 - recall: 0.8296 - val_loss: 1.5997 - val_accuracy: 0.7254 - val_precision: 0.7254 - val_recall: 1.0000 - val_f1: 0.8433 - val_roc_auc: 0.5000 - val_specificity: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 1s 72ms/step
 - val_f1: 0.904709748083242 - val_roc_auc: 0.7560615411743961 - val_specificity: 0.5471698113207547
312/312 [==============================] - 36s 114ms/step - loss: 0.2432 - accuracy: 0.9053 - precision: 0.9128 - recall: 0.8961 - val_loss: 0.3432 - val_accuracy: 0.8817 - val_precision: 0.8763 - val_recall: 0.9790 - val_f1: 0.9047 - val_roc_auc: 0.7561 - val_specificity: 0.5472
Epoch 3/10
10/10 [==============================] - 1s 76ms/step
 - val_f1: 0.9219512195121952 - val_roc_auc: 0.8975636278140247 - 

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

train_data_dir = "/content/BachelorData2/BachelorData/training"
validation_data_dir = "/content/BachelorData2/BachelorData/validation"
test_data_dir = "/content/BachelorData2/BachelorData/test"

# Define hyperparameters for the grid search
param_grid = {
    'learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64],
    'optimizer': [Adam, SGD, RMSprop],
    'epochs': [10, 15],
}

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Image dimensions
img_width, img_height = 200, 200
input_shape = (img_width, img_height, 3)

# Create EfficientNetB0 model with custom top layer
def create_efficientnet_model(optimizer, learning_rate):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    optimizer_instance = optimizer(learning_rate=learning_rate)
    model.compile(optimizer=optimizer_instance, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Grid search
best_score = 0
best_params = None

for params in ParameterGrid(param_grid):
    print(f"Training with parameters: {params}")

    # Create model
    model = create_efficientnet_model(params['optimizer'], params['learning_rate'])

    # Train the model
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=params['batch_size'],
        class_mode='binary')

    val_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=params['batch_size'],
        class_mode='binary',
        shuffle=False)

    model.fit(
        train_generator,
        epochs=params['epochs'],
        validation_data=val_generator)

    # Evaluate on the validation set
    val_preds = model.predict(val_generator)
    val_preds = np.where(val_preds > 0.5, 1, 0).flatten()
    val_true = val_generator.classes

    # Calculate F1 score
    val_f1_score = f1_score(val_true, val_preds)

    print(f"Validation F1 score: {val_f1_score}")

    if val_f1_score > best_score:
        best_score = val_f1_score
        best_params = params

print(f"Best parameters found: {best_params}")
print(f"Best validation F1 score: {best_score}")

# Create and train the model with the best parameters
best_model = create_efficientnet_model(best_params['optimizer'], best_params['learning_rate'])

best_train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=best_params['batch_size'],
    class_mode='binary')

best_val_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=best_params['batch_size'],
    class_mode='binary',
    shuffle=False)

best_model.fit(
    best_train_generator,
    epochs=best_params['epochs'],
    validation_data=best_val_generator)

# Evaluate on the test set
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=best_params['batch_size'],
    class_mode='binary',
    shuffle=False)

test_loss, test_accuracy = best_model.evaluate(test_generator)

test_preds = best_model.predict(test_generator)
test_preds = np.where(test_preds > 0.5, 1, 0).flatten()
test_true = test_generator.classes

test_precision = precision_score(test_true, test_preds)
test_recall = recall_score(test_true, test_preds)
test_f1_score = f1_score(test_true, test_preds)
test_auc_roc = roc_auc_score(test_true, test_preds)

print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Test F1 score: {test_f1_score}")
print(f"Test AUC-ROC: {test_auc_roc}")



Training with parameters: {'batch_size': 32, 'epochs': 10, 'learning_rate': 0.001, 'optimizer': <class 'keras.optimizers.adam.Adam'>}
16705208/16705208 [==============================] - 0s 0us/step
Found 20000 images belonging to 2 classes.
Found 587 images belonging to 2 classes.
Epoch 1/10
625/625 [==============================] - 198s 229ms/step - loss: 0.1474 - accuracy: 0.9440 - val_loss: 18.1694 - val_accuracy: 0.6031
Epoch 2/10
625/625 [==============================] - 141s 226ms/step - loss: 0.0815 - accuracy: 0.9711 - val_loss: 0.4843 - val_accuracy: 0.7376
Epoch 3/10
524/625 [========================>.....] - ETA: 22s - loss: 0.0609 - accuracy: 0.9779

KeyboardInterrupt: ignored